In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.preprocessing import StandardScaler 
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.externals import joblib
import time
from sklearn.decomposition import TruncatedSVD
from sklearn.model_selection import train_test_split
from sklearn import preprocessing

import warnings
warnings.simplefilter("ignore", category=DeprecationWarning)

In [ ]:
propiedades = pd.read_csv('../../set_datos_one_hot_encoder.csv')
set_pruebas = pd.read_csv('../set_pruebas_one_hot_encoder.csv')

columnas = ['superficie','lat', 'lon','property_type_encoded','seguridad','gimnasio', 'aire', \
            'pileta', 'cochera','transporte','comercio', 'servicios', 'doble piso', 'jardin']

In [ ]:
def encodear(x):
    return [float(x) for x in x.strip('[').strip(']').split(',')]

In [ ]:
propiedades.loc[:,'property_type_encoded'] = propiedades.loc[:,'property_type_encoded']\
.apply(lambda x: encodear(x))

set_pruebas.loc[:,'property_type_encoded'] = set_pruebas.loc[:,'property_type_encoded']\
.apply(lambda x: encodear(x))

In [ ]:
def predecir(x, estimador):
    x = list(x[:3]) + x[3] + list(x[4:])
    return estimador.predict([x])

In [ ]:
set_entrenamiento_datos = propiedades.loc[:,columnas]
set_entrenamiento_resultado = propiedades.loc[:,'price_aprox_usd']

In [ ]:
datos = []

for i in range(len(set_entrenamiento_datos)): 
    x = []
    for n in columnas:
        x.append(set_entrenamiento_datos[n][i])
    x = x[:3] + x[3] + x[4:]
    datos.append(x)

In [ ]:
def predecir(x, estimador):
    x = list(x[:3]) + x[3] + list(x[4:])
    return estimador.predict([x])

In [ ]:
datos = []

for i in range(len(set_entrenamiento_datos)): 
    x = []
    for n in columnas:
        x.append(set_entrenamiento_datos[n][i])
    x = x[:3] + x[3] + x[4:]
    datos.append(x)

# Decision tree con gridsearch

In [ ]:
criterios = ['mse']
max_depth = [None,150]
features = [0.8,1.0]

parametros = {"criterion" : criterios, "max_depth" : max_depth, "max_features" : features}

iteraciones_cross_validation = 5
arbol = DecisionTreeRegressor()

In [ ]:
inicio = time.strftime("%X")

grid = GridSearchCV( estimator = arbol, param_grid = parametros, n_jobs = 2, cv = iteraciones_cross_validation)

grid.fit(datos, set_entrenamiento_resultado)

score = grid.best_score_ * 100
mejores_parametros = grid.best_params_
fin = time.strftime("%X")

print("Tiempo: {} --- {} \n Precision: {:.2f} \n Parametros = {}".format(inicio,fin,score,mejores_parametros))

In [ ]:
set_pruebas.loc[:,'price_usd'] = set_pruebas.loc[:,columnas].apply(lambda x: predecir(x, grid)[0],axis = 1)

resultado = set_pruebas.loc[:,['id','price_usd']]

resultado.to_csv('resultados/Decision_tree_gridsearch_oneHotEncoder.csv', index = False)

# Extra trees con gridsearch

In [24]:
criterios = ['mse']
max_depth = [None,100]
features = [0.4,0.6,1.0]
estimators = [20, 30,50]

parametros = {"criterion" : criterios, "max_depth" : max_depth, \
              "n_estimators" : estimators, "max_features" : features}

iteraciones_cross_validation = 10
arbol = ExtraTreesRegressor()

In [25]:
inicio = time.strftime("%X")

grid = GridSearchCV( estimator = arbol, param_grid = parametros, n_jobs = 2, cv = iteraciones_cross_validation)

grid.fit(datos, set_entrenamiento_resultado)

score = grid.best_score_ * 100
mejores_parametros = grid.best_params_
fin = time.strftime("%X")

print("Tiempo: {} --- {} \n Precision: {:.2f} \n Parametros = {}".format(inicio,fin,score,mejores_parametros))

Tiempo: 17:21:24 --- 18:29:07 
 Precision: 97.57 
 Parametros = {'max_depth': 100, 'max_features': 0.4, 'n_estimators': 50, 'criterion': 'mse'}


In [26]:
set_pruebas.loc[:,'price_usd'] = set_pruebas.loc[:,columnas].apply(lambda x: predecir(x, grid)[0],axis = 1)

resultado = set_pruebas.loc[:,['id','price_usd']]

resultado.to_csv('resultados/Extra_trees_gridsearch_con_indice.csv', index = False)

# Usando el state name encoded

In [90]:
propiedades = pd.read_csv('../../set_datos_one_hot_encoder.csv')
set_pruebas = pd.read_csv('../set_pruebas_one_hot_encoder.csv')

columnas = ['superficie','lat','lon','property_type_encoded', 'state_name_encoded',\
            'seguridad','gimnasio','aire','pileta','cochera']

In [91]:
def encodear(x):
    return [float(x) for x in x.strip('[').strip(']').split(',')]

In [92]:
propiedades.loc[:,'property_type_encoded'] = propiedades.loc[:,'property_type_encoded']\
.apply(lambda x: encodear(x))

propiedades.loc[:,'state_name_encoded'] = propiedades.loc[:,'state_name_encoded']\
.apply(lambda x: encodear(x))

set_pruebas.loc[:,'property_type_encoded'] = set_pruebas.loc[:,'property_type_encoded']\
.apply(lambda x: encodear(x))

set_pruebas.loc[:,'state_name_encoded'] = set_pruebas.loc[:,'state_name_encoded']\
.apply(lambda x: encodear(x))

In [93]:
def predecir(x, estimador):
    x = list(x[:3]) + x[3] + x[4] + list(x[5:])
    return estimador.predict([x])

In [94]:
set_entrenamiento_datos = propiedades.loc[:,columnas]
set_entrenamiento_resultado = propiedades.loc[:,'price_aprox_usd']

In [95]:
datos = []

for i in range(len(set_entrenamiento_datos)): 
    x = []
    for n in columnas:
        x.append(set_entrenamiento_datos[n][i])
    x = x[:3] + x[3] + x[4] + x[5:]
    datos.append(x)

In [96]:
criterios = ['mse']
max_depth = [None,100,150]
features = [0.6,0.8,1.0]

parametros = {"criterion" : criterios, "max_depth" : max_depth, "max_features" : features}

iteraciones_cross_validation = 5
arbol = DecisionTreeRegressor()

In [97]:
inicio = time.strftime("%X")

grid = GridSearchCV( estimator = arbol, param_grid = parametros, n_jobs = 2, cv = iteraciones_cross_validation)

grid.fit(datos, set_entrenamiento_resultado)

score = grid.best_score_ * 100
mejores_parametros = grid.best_params_
fin = time.strftime("%X")

print("Tiempo: {} --- {} \n Precision: {:.2f} \n Parametros = {}".format(inicio,fin,score,mejores_parametros))

Tiempo: 03:06:31 --- 03:10:57 
 Precision: 96.16 
 Parametros = {'criterion': 'mse', 'max_depth': 100, 'max_features': 0.6}


In [98]:
set_pruebas.loc[:,'price_usd'] = set_pruebas.loc[:,columnas].apply(lambda x: predecir(x, grid)[0],axis = 1)

resultado = set_pruebas.loc[:,['id','price_usd']]

resultado.to_csv('resultados/Decision_tree_HotEncoder_StateName.csv', index = False)

# Aplicando Bagging

In [ ]:
propiedades = pd.read_csv('../../set_datos_one_hot_encoder.csv')
set_pruebas = pd.read_csv('../set_pruebas_one_hot_encoder.csv')

In [ ]:
def encodear(x):
    return [float(x) for x in x.strip('[').strip(']').split(',')]

In [ ]:
propiedades.loc[:,'property_type_encoded'] = propiedades.loc[:,'property_type_encoded']\
.apply(lambda x: encodear(x))

set_pruebas.loc[:,'property_type_encoded'] = set_pruebas.loc[:,'property_type_encoded']\
.apply(lambda x: encodear(x))

In [ ]:
def predecir(x, estimador):
    x = list(x[:3]) + x[3] + list(x[4:])
    return estimador.predict([x])

In [ ]:
set_pruebas.loc[:,'price_usd'] = 0.0

cant = 10
col = ['superficie','lat', 'lon','property_type_encoded']
comodidades = ['seguridad','gimnasio','aire','pileta','cochera','transporte','comercio','servicios','doble piso','jardin']

for i in range(cant):
    com = list(np.random.choice(comodidades,7,replace = False))
    columnas = col + com
    set_entrenamiento_datos = propiedades.loc[:,columnas]
    set_entrenamiento_resultado = propiedades.loc[:,'price_aprox_usd']
    
    datos = []

    for k in range(len(set_entrenamiento_datos)): 
        x = []
        for n in columnas:
            x.append(set_entrenamiento_datos[n][k])
        x = x[:3] + x[3] + x[4:]
        datos.append(x)
    
    tree = DecisionTreeRegressor(criterion = 'mse', max_depth = 100, max_features = 0.8)
    
    tree.fit(datos,set_entrenamiento_resultado)
    set_pruebas.loc[:,'res'] = set_pruebas.loc[:,columnas].apply(lambda x: predecir(x, tree)[0],axis = 1)
    set_pruebas.loc[:,'price_usd'] = set_pruebas.loc[:,'price_usd'] + set_pruebas.loc[:,'res']

In [ ]:
set_pruebas.loc[:,'price_usd'] = set_pruebas.loc[:,'price_usd'] / cant

resultado = set_pruebas.loc[:,['id','price_usd']]

resultado.to_csv('resultados/Decision_tree_bagging_OneHotEncoder.csv', index = False)

# Realizando bagging de a dos en dos

In [ ]:
propiedades = pd.read_csv('../../set_datos_one_hot_encoder.csv')
set_pruebas = pd.read_csv('../set_pruebas_one_hot_encoder.csv')

In [ ]:
def encodear(x):
    return [float(x) for x in x.strip('[').strip(']').split(',')]

In [ ]:
propiedades.loc[:,'property_type_encoded'] = propiedades.loc[:,'property_type_encoded']\
.apply(lambda x: encodear(x))

set_pruebas.loc[:,'property_type_encoded'] = set_pruebas.loc[:,'property_type_encoded']\
.apply(lambda x: encodear(x))

In [ ]:
def predecir(x, estimador):
    x = list(x[:3]) + x[3] + list(x[4:])
    return estimador.predict([x])

In [ ]:
set_pruebas.loc[:,'price_usd'] = 0.0

cant = 10
col = ['superficie','lat', 'lon','property_type_encoded']
comodidades = ['seguridad','gimnasio','aire','pileta','cochera','transporte','comercio','servicios','doble piso','jardin']

for i in range(cant):
    com = list(np.random.choice(comodidades,7,replace = False))
    columnas = col + com
    set_entrenamiento_datos = propiedades.loc[:,columnas]
    set_entrenamiento_resultado = propiedades.loc[:,'price_aprox_usd']

    datos = []

    for k in range(len(set_entrenamiento_datos)): 
        x = []
        for n in columnas:
            x.append(set_entrenamiento_datos[n][k])
        x = x[:3] + x[3] + x[4:]
        datos.append(x)

    tree = DecisionTreeRegressor(criterion = 'mse', max_depth = 100, max_features = 0.8)

    tree.fit(datos,set_entrenamiento_resultado)
    set_pruebas.loc[:,'price_usd'] = (set_pruebas.loc[:,columnas].apply(lambda x: predecir(x, tree)[0],axis = 1) + \
            set_pruebas.loc[:,'price_usd']) / 2
resultado = set_pruebas.loc[:,['id','price_usd']]

resultado.to_csv('resultados/Decision_tree_bagging_OneHotEncoder_2En2.csv', index = False)